# <img align="left" src="./images/movie_camera.png"     style=" width:40px;  " > ห้องปฏิบัติการฝึกหัด: ระบบแนะนำแบบ Collaborative Filtering (Collaborative Filtering Recommender Systems)

In this exercise, you will implement collaborative filtering to build a recommender system for movies. 

# <img align="left" src="./images/film_reel.png"     style=" width:40px;  " > Outline
- [ 1 - สัญลักษณ์](#1)
- [ 2 - ระบบแนะนำ](#2)
- [ 3 - ชุดข้อมูลการให้คะแนนภาพยนตร์](#3)
- [ 4 - อัลกอริทึมการเรียนรู้แบบ Collaborative Filtering](#4)
  - [ 4.1 ฟังก์ชันต้นทุนของ Collaborative Filtering](#4.1)
    - [ แบบฝึกหัด 1](#ex01)
- [ 5 - การเรียนรู้คำแนะนำภาพยนตร์](#5)
- [ 6 - คำแนะนำ (Recommendations) ](#6)
- [ 7 - ยินดีด้วย!](#7)




##  แพ็คเกจ (Packages) <img align="left" src="./images/film_strip_vertical.png"     style=" width:40px;   " >
เราจะใช้ NumPy และ TensorFlow ซึ่งเป็นแพ็คเกจที่คุ้นเคยกันดี

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()
try:
  %matplotlib widget
  print("widget is already installed")
except:
  print("widget is not been installed, install now..")
  !pip install ipympl

In [ ]:
!git clone https://github.com/Smith-WeStrideTH/Unsupervised_Learning_Course.git
%cd Unsupervised_Learning_Course/work 

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from recsys_utils import *

<a name="1"></a>
## 1 - สัญลักษณ์


|General <br />  Notation  | Description| Python (if any) |
|:-------------|:------------------------------------------------------------||
| $r(i,j)$     | scalar; = 1  if user j rated movie i  = 0  otherwise             ||
| $y(i,j)$     | scalar; = rating given by user j on movie  i    (if r(i,j) = 1 is defined) ||
|$\mathbf{w}^{(j)}$ | vector; parameters for user j ||
|$b^{(j)}$     |  scalar; parameter for user j ||
| $\mathbf{x}^{(i)}$ |   vector; feature ratings for movie i        ||     
| $n_u$        | number of users |num_users|
| $n_m$        | number of movies | num_movies |
| $n$          | number of features | num_features                    |
| $\mathbf{X}$ |  matrix of vectors $\mathbf{x}^{(i)}$         | X |
| $\mathbf{W}$ |  matrix of vectors $\mathbf{w}^{(j)}$         | W |
| $\mathbf{b}$ |  vector of bias parameters $b^{(j)}$ | b |
| $\mathbf{R}$ | matrix of elements $r(i,j)$                    | R |



<a name="2"></a>
## 2 - ระบบแนะนำ <img align="left" src="./images/film_rating.png" style=" width:40px;  " >
ในห้องปฏิบัติการนี้ คุณจะได้เรียนรู้การนำไปใช้ของอัลกอริทึมการกรองแบบร่วมมือ และนำไปใช้กับชุดข้อมูลการให้คะแนนภาพยนตร์

เป้าหมายของระบบการแนะนำแบบกรองความร่วมมือคือการสร้างเวกเตอร์สองตัว: สำหรับผู้ใช้แต่ละคน เวกเตอร์พารามิเตอร์ที่สะท้อนถึงรสนิยมภาพยนตร์ของผู้ใช้ สำหรับแต่ละภาพยนตร์ เวกเตอร์ฟีเจอร์ที่มีขนาดเท่ากันซึ่งสะท้อนถึงคำอธิบายบางอย่างของภาพยนตร์ ผลคูณดอทของสองเวกเตอร์บวกกับเทอมอคติควรสร้างค่าประมาณของคะแนนที่ผู้ใช้อาจให้กับภาพยนตร์นั้น

แผนภาพด้านล่างแสดงรายละเอียดวิธีการเรียนรู้เวกเตอร์เหล่านี้

<figure>
   <img src="./images/ColabFilterLearn.PNG"  style="width:740px;height:250px;" >
</figure>

เรามีข้อมูลการให้คะแนนในรูปแบบเมทริกซ์ดังแสดง $Y$ ประกอบด้วยคะแนนตั้งแต่ 0.5 ถึง 5 โดยเพิ่มทีละ 0.5 คะแนน และ 0 หากภาพยนตร์ยังไม่ได้รับการให้คะแนน  $R$  มีค่า 1 เมื่อภาพยนตร์ได้รับการให้คะแนน ภาพยนตร์อยู่ในแถว ผู้ใช้ในคอลัมน์ แต่ละผู้ใช้มีเวกเตอร์พารามิเตอร์  $w^{user}$ และไบแอส แต่ละภาพยนตร์มีเวกเตอร์ฟีเจอร์  $x^{movie}$เวกเตอร์เหล่านี้ถูกเรียนรู้พร้อมกันโดยใช้การให้คะแนนผู้ใช้/ภาพยนตร์ที่มีอยู่เป็นข้อมูลการฝึก ตัวอย่างการฝึกหนึ่งแสดงไว้ด้านบน:  $\mathbf{w}^{(1)} \cdot \mathbf{x}^{(1)} + b^{(1)} = 4$ ควรสังเกตว่าเวกเตอร์ฟีเจอร์ $x^{movie}$ ต้องตอบสนองผู้ใช้ทั้งหมด ในขณะที่เวกเตอร์ผู้ใช้  $w^{user}$ ต้องตอบสนองภาพยนตร์ทั้งหมด นี่คือที่มาของชื่อวิธีการนี้ - ผู้ใช้ทุกคนร่วมมือกันสร้างชุดการให้คะแนน"

<figure>
   <img src="./images/ColabFilterUse.PNG"  style="width:640px;height:250px;" >
</figure>

เมื่อเรียนรู้เวกเตอร์คุณลักษณะและพารามิเตอร์แล้ว สามารถนำไปใช้ในการทำนายคะแนนที่ผู้ใช้คนหนึ่งอาจให้กับภาพยนตร์ที่ยังไม่ได้ให้คะแนนได้ ดังแสดงในแผนภาพด้านบน สมการเป็นตัวอย่างของการทำนายคะแนนสำหรับผู้ใช้ที่หนึ่งสำหรับภาพยนตร์ที่ศูนย์


ในแบบฝึกหัดนี้ คุณจะได้ฝึกใชฟังก์ชัน `cofiCostFunc` ที่คำนวณฟังก์ชันวัตถุประสงค์ของการกรองร่วมกัน หลังจากนำฟังก์ชันวัตถุประสงค์ไปใช้แล้ว คุณจะใช้ลูปการฝึกอบรมแบบกำหนดเองของ TensorFlow เพื่อเรียนรู้พารามิเตอร์สำหรับการกรองร่วมกัน ขั้นตอนแรกคือการรายละเอียดชุดข้อมูลและโครงสร้างข้อมูลที่จะใช้ในห้องปฏิบัติการ

<a name="3"></a>
## 3 - ชุดข้อมูลการให้คะแนนภาพยนตร์ <img align="left" src="./images/film_rating.png"     style=" width:40px;  " >
ชุดข้อมูลนี้มาจาก [MovieLens "ml-latest-small"](https://grouplens.org/datasets/movielens/latest/).   
[F. Maxwell Harper and Joseph A. Konstan. 2015. The MovieLens Datasets: History and Context. ACM Transactions on Interactive Intelligent Systems (TiiS) 5, 4: 19:1–19:19. <https://doi.org/10.1145/2827872>]

ชุดข้อมูลเดิมมีภาพยนตร์ 9,000 เรื่องที่ได้รับการจัดอันดับโดยผู้ใช้ 600 คน ชุดข้อมูลได้ถูกปรับขนาดให้เน้นภาพยนตร์ตั้งแต่ปี 2000 เป็นต้นไป ชุดข้อมูลที่ลดขนาดแล้วมี $n_u = 443$ ผู้ใช้ และ $n_m= 4778$ ภาพยนตร์ 

ด้านล่าง คุณจะโหลดชุดข้อมูลภาพยนตร์ลงในตัวแปร  $Y$ และ  $R$.

เมทริกซ์  $Y$ (a  $n_m \times n_u$ เมทริกซ์) เก็บคะแนน  $y^{(i,j)}$. เมทริกซ์ $R$ เป็นเมทริกซ์ตัวบ่งชี้ค่าไบนารี โดยที่  $R(i,j) = 1$ ถ้าผู้ใช้ $j$ ให้คะแนนกับภาพยนตร์ $i$, และ   $R(i,j)=0$ มิฉะนั้น

ตลอดส่วนนี้ของแบบฝึกหัด คุณจะได้ทำงานกับเมทริกซ์, $\mathbf{X}$, $\mathbf{W}$ และ  $\mathbf{b}$ ด้วย: 

$$\mathbf{X} = 
\begin{bmatrix}
--- (\mathbf{x}^{(0)})^T --- \\
--- (\mathbf{x}^{(1)})^T --- \\
\vdots \\
--- (\mathbf{x}^{(n_m-1)})^T --- \\
\end{bmatrix} , \quad
\mathbf{W} = 
\begin{bmatrix}
--- (\mathbf{w}^{(0)})^T --- \\
--- (\mathbf{w}^{(1)})^T --- \\
\vdots \\
--- (\mathbf{w}^{(n_u-1)})^T --- \\
\end{bmatrix},\quad
\mathbf{ b} = 
\begin{bmatrix}
 b^{(0)}  \\
 b^{(1)} \\
\vdots \\
b^{(n_u-1)} \\
\end{bmatrix}\quad
$$ 

แถวที่  $i$-th ของ  $\mathbf{X}$ สอดคล้องกับเวกเตอร์ฟีเจอร์ $x^{(i)}$ สำหรับภาพยนตร์ที่ $i$-th และแถวที่  $j$-th ของ
$\mathbf{W}$ สอดคล้องกับเวกเตอร์พารามิเตอร์หนึ่งตัว $\mathbf{w}^{(j)}$, สำหรับผู้ใช้ที่ 
$j$-th ทั้ง $x^{(i)}$ และ  $\mathbf{w}^{(j)}$ เป็นเวกเตอร์ $n$-มิติ 
สำหรับวัตถุประสงค์ของแบบฝึกหัดนี้ คุณจะใช้ $n=10$, และดังนั้น $\mathbf{x}^{(i)}$ และ $\mathbf{w}^{(j)}$ มี 10 องค์ประกอบ.
และดังนั้น, $\mathbf{X}$ คือเมทริกซ์
$n_m \times 10$ เมทริกซ์ และ $\mathbf{W}$ คือเมทริกซ์ $n_u \times 10$.

เราจะเริ่มต้นด้วยการโหลดชุดข้อมูลการให้คะแนนภาพยนตร์เพื่อทำความเข้าใจโครงสร้างของข้อมูล 
เราจะโหลด $Y$ และ  $R$ ด้วยชุดข้อมูลภาพยนตร์ 
เราจะโหลด $\mathbf{X}$, $\mathbf{W}$, และ  $\mathbf{b}$  ด้วยค่าที่คำนวณไว้ล่วงหน้า ค่าเหล่านี้จะถูกเรียนรู้ในภายหลังในห้องปฏิบัติการ แต่เราจะใช้ค่าที่คำนวณไว้ล่วงหน้าเพื่อพัฒนาโมเดลต้นทุน

In [ ]:
#Load data
X, W, b, num_movies, num_features, num_users = load_precalc_params_small()
Y, R = load_ratings_small()

print("Y", Y.shape, "R", R.shape)
print("X", X.shape)
print("W", W.shape)
print("b", b.shape)
print("num_features", num_features)
print("num_movies",   num_movies)
print("num_users",    num_users)

In [ ]:
#  From the matrix, we can compute statistics like average rating.
tsmean =  np.mean(Y[0, R[0, :].astype(bool)])
print(f"Average rating for movie 1 : {tsmean:0.3f} / 5" )

<a name="4"></a>
## 4 - อัลกอริทึมการเรียนรู้แบบ Collaborative Filtering <img align="left" src="./images/film_filter.png"     style=" width:40px;  " >

ตอนนี้คุณจะเริ่มต้นการนำไปใช้ของอัลกอริทึมการเรียนรู้แบบ Collaborative Filtering คุณจะเริ่มต้นด้วยการนำฟังก์ชันวัตถุประสงค์มาใช้

อัลกอริทึมการกรองแบบร่วมมือกันในการตั้งค่าของคำแนะนำภาพยนตร์พิจารณาชุดของเวกเตอร์พารามิเตอร์ $n$-มิติ
$\mathbf{x}^{(0)},...,\mathbf{x}^{(n_m-1)}$, $\mathbf{w}^{(0)},...,\mathbf{w}^{(n_u-1)}$ and $b^{(0)},...,b^{(n_u-1)}$, where the
โดยที่โมเดลทำนายการให้คะแนนสำหรับภาพยนตร์  $i$ โดยผู้ใช้ r $j$ เป็น
$y^{(i,j)} = \mathbf{w}^{(j)}\cdot \mathbf{x}^{(i)} + b^{(j)}$ . เมื่อได้รับชุดข้อมูลที่ประกอบด้วย
ชุดของการให้คะแนนที่สร้างโดยผู้ใช้บางคนในภาพยนตร์บางเรื่อง คุณต้องการเรียนรู้
เวกเตอร์พารามิเตอร์  $\mathbf{x}^{(0)},...,\mathbf{x}^{(n_m-1)},
\mathbf{w}^{(0)},...,\mathbf{w}^{(n_u-1)}$  and $b^{(0)},...,b^{(n_u-1)}$  ที่สร้างฟิตที่ดีที่สุด ลด
ค่าความคลาดเคลื่อนกำลังสอง (minimizesthe squared error).

คุณจะเติมโค้ดใน cofiCostFunc เพื่อคำนวณฟังก์ชันต้นทุนสำหรับการกรองแบบร่วมมือกัน


<a name="4.1"></a>
### 4.1 ฟังก์ชันต้นทุนของ Collaborative Filtering

ฟังก์ชันต้นทุนการกรองแบบร่วมมือกันกำหนดโดย
$$J({\mathbf{x}^{(0)},...,\mathbf{x}^{(n_m-1)},\mathbf{w}^{(0)},b^{(0)},...,\mathbf{w}^{(n_u-1)},b^{(n_u-1)}})= \left[ \frac{1}{2}\sum_{(i,j):r(i,j)=1}(\mathbf{w}^{(j)} \cdot \mathbf{x}^{(i)} + b^{(j)} - y^{(i,j)})^2 \right]
+ \underbrace{\left[
\frac{\lambda}{2}
\sum_{j=0}^{n_u-1}\sum_{k=0}^{n-1}(\mathbf{w}^{(j)}_k)^2
+ \frac{\lambda}{2}\sum_{i=0}^{n_m-1}\sum_{k=0}^{n-1}(\mathbf{x}_k^{(i)})^2
\right]}_{regularization}
\tag{1}$$
ผลรวมแรกใน (1) คือ "สำหรับทุก $i$, $j$ โดยที่  $r(i,j)$ เท่ากับ  $1$" และสามารถเขียนได้ดังนี้:

$$
= \left[ \frac{1}{2}\sum_{j=0}^{n_u-1} \sum_{i=0}^{n_m-1}r(i,j)*(\mathbf{w}^{(j)} \cdot \mathbf{x}^{(i)} + b^{(j)} - y^{(i,j)})^2 \right]
+\text{regularization}
$$

ตอนนี้คุณควรเขียน cofiCostFunc (ฟังก์ชันต้นทุนการกรองแบบร่วมมือกัน) เพื่อส่งคืนต้นทุนนี้

<a name="ex01"></a>
### แบบฝึกหัด 1

**การใช้ Loop แบบ For:**   
เริ่มต้นด้วยการนำฟังก์ชันต้นทุนไปใช้งานโดยใช้ลูป for
พิจารณาพัฒนาฟังก์ชันต้นทุนในสองขั้นตอน ขั้นแรก พัฒนาฟังก์ชันต้นทุนโดยไม่ใช้การปรับเทียบ กรณีทดสอบที่ไม่รวมการปรับเทียบมีให้ด้านล่างเพื่อทดสอบการใช้งานของคุณ เมื่อใช้งานได้แล้ว ให้เพิ่มการปรับเทียบและเรียกใช้การทดสอบที่รวมการปรับเทียบ โปรดทราบว่าคุณควรสะสมต้นทุนสำหรับผู้ใช้ $j$ และภาพยนตร์ $i$ เฉพาะเมื่อ $R(i,j) = 1$.

In [ ]:
# GRADED FUNCTION: cofi_cost_func
# UNQ_C1

def cofi_cost_func(X, W, b, Y, R, lambda_):
    """
    Returns the cost for the content-based filtering
    Args:
      X (ndarray (num_movies,num_features)): matrix of item features
      W (ndarray (num_users,num_features)) : matrix of user parameters
      b (ndarray (1, num_users)            : vector of user parameters
      Y (ndarray (num_movies,num_users)    : matrix of user ratings of movies
      R (ndarray (num_movies,num_users)    : matrix, where R(i, j) = 1 if the i-th movies was rated by the j-th user
      lambda_ (float): regularization parameter
    Returns:
      J (float) : Cost
    """
    nm, nu = Y.shape
    J = 0
    ### START CODE HERE ###  
    
        
        
        
            
            
            
            
    
    
    ### END CODE HERE ### 

    return J

<details>
  <summary><font size="3" color="darkgreen"><b>Click for hints</b></font></summary>
    คุณสามารถแบ่งโครงสร้างโค้ดเป็นสองฟังก์ชันวนซ้ำได้ คล้ายกับการหาผลรวมใน (1)
    สร้างโค้ดโดยไม่ต้องมีการปรับค่า regularization ก่อน
    โปรดสังเกตว่าบางองค์ประกอบใน (1) เป็นเวกเตอร์ ให้นำ np.dot() มาใช้ คุณยังสามารถใช้ np.square() ได้อีกด้วย
    ใส่ใจเป็นพิเศษกับตัวแปรดัชนี โดยแยกแยะว่าองค์ประกอบใดถูกดัชนีด้วย i และองค์ประกอบใดถูกดัชนีด้วย j อย่าลืมหารด้วยสอง    

```python     
    ### START CODE HERE ###  
    for j in range(nu):
        
        
        for i in range(nm):
            
            
    ### END CODE HERE ### 
```    
<details>
    <summary><font size="2" color="darkblue"><b> ดูคำแนะนำเพิ่มเติม</b></font></summary>
        
    นี่คือรายละเอียดเพิ่มเติม
    โค้ดด้านล่างดึงแต่ละองค์ประกอบจากเมทริกซ์ก่อนนำไปใช้
    นอกจากนี้ยังสามารถอ้างอิงเมทริกซ์โดยตรงได้อีกด้วย
    โค้ดนี้ไม่มีการปรับค่า regularization
    
```python 
    nm,nu = Y.shape
    J = 0
    ### START CODE HERE ###  
    for j in range(nu):
        w = W[j,:]
        b_j = b[0,j]
        for i in range(nm):
            x = 
            y = 
            r =
            J += 
    J = J/2
    ### END CODE HERE ### 

```
    
<details>
    <summary><font size="2" color="darkblue"><b> ทางเลือกสุดท้าย (full non-regularized implementation)</b></font></summary>
    
```python 
    nm,nu = Y.shape
    J = 0
    ### START CODE HERE ###  
    for j in range(nu):
        w = W[j,:]
        b_j = b[0,j]
        for i in range(nm):
            x = X[i,:]
            y = Y[i,j]
            r = R[i,j]
            J += np.square(r * (np.dot(w,x) + b_j - y ) )
    J = J/2
    ### END CODE HERE ### 
```
    
<details>
    <summary><font size="2" color="darkblue"><b>regularization</b></font></summary>
     การปรับค่าพารามิเตอร์ (Regularization) จะทำการยกกำลังสองแต่ละองค์ประกอบของอาร์เรย์ W และ X จากนั้นนำผลลัพธ์ทั้งหมดมาบวกกัน
    คุณสามารถใช้ np.square() และ np.sum() เพื่อคำนวณได้

<details>
    <summary><font size="2" color="darkblue"><b>รายละเอียด regularization </b></font></summary>
    
```python 
    J += (lambda_/2) * (np.sum(np.square(W)) + np.sum(np.square(X)))
```
    
</details>
</details>
</details>
</details>

    


In [ ]:
# Reduce the data set size so that this runs faster
num_users_r = 4
num_movies_r = 5 
num_features_r = 3

X_r = X[:num_movies_r, :num_features_r]
W_r = W[:num_users_r,  :num_features_r]
b_r = b[0, :num_users_r].reshape(1,-1)
Y_r = Y[:num_movies_r, :num_users_r]
R_r = R[:num_movies_r, :num_users_r]

# Evaluate cost function
J = cofi_cost_func(X_r, W_r, b_r, Y_r, R_r, 0);
print(f"Cost: {J:0.2f}")

**ผลลัพธ์คาดหวัง (lambda = 0)**:  
$13.67$.

In [ ]:
# Evaluate cost function with regularization 
J = cofi_cost_func(X_r, W_r, b_r, Y_r, R_r, 1.5);
print(f"Cost (with regularization): {J:0.2f}")

**ผลลัพธ์คาดหวัง**:

28.09

In [ ]:
# Public tests
from public_tests_r import *
test_cofi_cost_func(cofi_cost_func)

**การนำไปใช้แบบเวกเตอร์ (Vectorized Implementation)**

เป็นสิ่งสำคัญในการสร้างการนำไปใช้แบบเวกเตอร์เพื่อคำนวณ $J$, เนื่องจากจะถูกเรียกใช้หลายครั้งในระหว่างการปรับให้เหมาะสม พีชคณิตเชิงเส้นที่ใช้ไม่ได้เป็นจุดสนใจหลักของชุดนี้ ดังนั้นจึงมีการให้การนำไปใช้ หากคุณเป็นผู้เชี่ยวชาญด้านพีชคณิตเชิงเส้น คุณสามารถสร้างเวอร์ชันของคุณเองโดยไม่ต้องอ้างอิงโค้ดด้านล่าง

เรียกใช้โค้ดด้านล่างและตรวจสอบว่าให้ผลลัพธ์เหมือนกับเวอร์ชันที่ไม่ใช่เวกเตอร์

In [ ]:
def cofi_cost_func_v(X, W, b, Y, R, lambda_):
    """
    Returns the cost for the content-based filtering
    Vectorized for speed. Uses tensorflow operations to be compatible with custom training loop.
    Args:
      X (ndarray (num_movies,num_features)): matrix of item features
      W (ndarray (num_users,num_features)) : matrix of user parameters
      b (ndarray (1, num_users)            : vector of user parameters
      Y (ndarray (num_movies,num_users)    : matrix of user ratings of movies
      R (ndarray (num_movies,num_users)    : matrix, where R(i, j) = 1 if the i-th movies was rated by the j-th user
      lambda_ (float): regularization parameter
    Returns:
      J (float) : Cost
    """
    j = (tf.linalg.matmul(X, tf.transpose(W)) + b - Y)*R
    J = 0.5 * tf.reduce_sum(j**2) + (lambda_/2) * (tf.reduce_sum(X**2) + tf.reduce_sum(W**2))
    return J

In [ ]:
# Evaluate cost function
J = cofi_cost_func_v(X_r, W_r, b_r, Y_r, R_r, 0);
print(f"Cost: {J:0.2f}")

# Evaluate cost function with regularization 
J = cofi_cost_func_v(X_r, W_r, b_r, Y_r, R_r, 1.5);
print(f"Cost (with regularization): {J:0.2f}")

**ผลลัพธ์คาดหวัง**:  
Cost: 13.67  
Cost (with regularization): 28.09

<a name="5"></a>
## 5 - การเรียนรู้คำแนะนำภาพยนตร์ <img align="left" src="./images/film_man_action.png" style=" width:40px;  " >
------------------------------

หลังจากที่คุณเสร็จสิ้นการนำฟังก์ชันต้นทุนการกรองแบบร่วมมือกันไปใช้งานแล้ว คุณสามารถเริ่มฝึกอัลกอริทึมของคุณเพื่อสร้างคำแนะนำภาพยนตร์สำหรับตัวคุณเองได้

ในเซลล์ด้านล่าง คุณสามารถป้อนตัวเลือกภาพยนตร์ของคุณเอง อัลกอริทึมจะทำการแนะนำภาพยนตร์ให้กับคุณ! เราได้กรอกค่าบางค่าตามความชอบของเรา แต่หลังจากที่คุณใช้งานได้แล้ว คุณควรเปลี่ยนสิ่งนี้ให้ตรงกับรสนิยมของคุณ รายชื่อภาพยนตร์ทั้งหมดในชุดข้อมูลอยู่ในไฟล์[movie list](data/small_movie_list.csv).

In [ ]:
movieList, movieList_df = load_Movie_List_pd()

my_ratings = np.zeros(num_movies)          #  Initialize my ratings

# Check the file small_movie_list.csv for id of each movie in our dataset
# For example, Toy Story 3 (2010) has ID 2700, so to rate it "5", you can set
my_ratings[2700] = 5 

#Or suppose you did not enjoy Persuasion (2007), you can set
my_ratings[2609] = 2;

# We have selected a few movies we liked / did not like and the ratings we
# gave are as follows:
my_ratings[929]  = 5   # Lord of the Rings: The Return of the King, The
my_ratings[246]  = 5   # Shrek (2001)
my_ratings[2716] = 3   # Inception
my_ratings[1150] = 5   # Incredibles, The (2004)
my_ratings[382]  = 2   # Amelie (Fabuleux destin d'Amélie Poulain, Le)
my_ratings[366]  = 5   # Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)
my_ratings[622]  = 5   # Harry Potter and the Chamber of Secrets (2002)
my_ratings[988]  = 3   # Eternal Sunshine of the Spotless Mind (2004)
my_ratings[2925] = 1   # Louis Theroux: Law & Disorder (2008)
my_ratings[2937] = 1   # Nothing to Declare (Rien à déclarer)
my_ratings[793]  = 5   # Pirates of the Caribbean: The Curse of the Black Pearl (2003)
my_rated = [i for i in range(len(my_ratings)) if my_ratings[i] > 0]

print('\nNew user ratings:\n')
for i in range(len(my_ratings)):
    if my_ratings[i] > 0 :
        print(f'Rated {my_ratings[i]} for  {movieList_df.loc[i,"title"]}');

ตอนนี้ มาเพิ่มรีวิวเหล่านี้ลงใน $Y$ และ  $R$ และทำให้คะแนนเป็นมาตรฐานกันเถอะ

In [ ]:
# Reload ratings
Y, R = load_ratings_small()

# Add new user ratings to Y 
Y = np.c_[my_ratings, Y]

# Add new user indicator matrix to R
R = np.c_[(my_ratings != 0).astype(int), R]

# Normalize the Dataset
Ynorm, Ymean = normalizeRatings(Y, R)

มาเตรียมการฝึกโมเดลกัน โดยเริ่มจากการเริ่มต้นพารามิเตอร์และเลือกออปติไมเซอร์ Adam

In [ ]:
#  Useful Values
num_movies, num_users = Y.shape
num_features = 100

# Set Initial Parameters (W, X), use tf.Variable to track these variables
tf.random.set_seed(1234) # for consistent results
W = tf.Variable(tf.random.normal((num_users,  num_features),dtype=tf.float64),  name='W')
X = tf.Variable(tf.random.normal((num_movies, num_features),dtype=tf.float64),  name='X')
b = tf.Variable(tf.random.normal((1,          num_users),   dtype=tf.float64),  name='b')

# Instantiate an optimizer.
optimizer = keras.optimizers.Adam(learning_rate=1e-1)

ตอนนี้เรามาฝึกโมเดลการกรองแบบร่วมมือกัน (collaborative filtering) กัน พารามิเตอร์ที่เราจะเรียนรู้คือ $\mathbf{X}$, $\mathbf{W}$, และ  $\mathbf{b}$. 

การดำเนินการที่เกี่ยวข้องกับการเรียนรู้ $w$, $b$, และ  $x$  พร้อมกันไม่ได้อยู่ใน 'เลเยอร์' แบบทั่วไปที่เสนอในแพ็คเกจเครือข่ายประสาทเทียมของ TensorFlow ดังนั้น โฟลว์ที่ใช้ใน Course 2: Model(), Compile(), Fit(), Predict() จึงไม่สามารถนำไปใช้ได้โดยตรง แต่เราสามารถใช้ลูปการฝึกอบรมแบบกำหนดเองได้

นึกย้อนกลับไปถึงขั้นตอนของการไล่ระดับ (gradient descent)จากแล็บก่อนหน้านี้:
- ทำซ้ำจนกว่าจะ converge:
    - คำนวณการส่งต่อไปข้างหน้า (forward)
    - คำนวณอนุพันธ์ (derivative) ของการสูญเสียเทียบกับพารามิเตอร์
    - อัปเดตพารามิเตอร์โดยใช้อัตราการเรียนรู้และอนุพันธ์ที่คำนวณได้

    
TensorFlow มีความสามารถที่ยอดเยี่ยมในการคำนวณอนุพันธ์ให้คุณ ซึ่งแสดงไว้ด้านล่าง ภายในส่วน  `tf.GradientTape()` TensorFlow มีความสามารถที่ยอดเยี่ยมในการคำนวณอนุพันธ์ให้คุณ ซึ่งแสดงไว้ด้านล่าง ภายในส่วน `tape.gradient()` ภายหลัง จะส่งคืนการไล่ระดับสีของการสูญเสียเทียบกับตัวแปรที่ติดตาม การไล่ระดับสีสามารถนำไปใช้กับพารามิเตอร์โดยใช้ออปติไมเซอร์

นี่คือการแนะนำสั้น ๆ เกี่ยวกับคุณสมบัติที่มีประโยชน์ของ TensorFlow และเฟรมเวิร์กการเรียนรู้ของเครื่องอื่นๆ สามารถค้นหาข้อมูลเพิ่มเติมได้โดยการสืบค้น "ลูปการฝึกอบรมแบบกำหนดเอง" ภายในเฟรมเวิร์กที่สนใจ
    


In [ ]:
iterations = 200
lambda_ = 1
for iter in range(iterations):
    # Use TensorFlow’s GradientTape
    # to record the operations used to compute the cost 
    with tf.GradientTape() as tape:

        # Compute the cost (forward pass included in cost)
        cost_value = cofi_cost_func_v(X, W, b, Ynorm, R, lambda_)

    # Use the gradient tape to automatically retrieve
    # the gradients of the trainable variables with respect to the loss
    grads = tape.gradient( cost_value, [X,W,b] )

    # Run one step of gradient descent by updating
    # the value of the variables to minimize the loss.
    optimizer.apply_gradients( zip(grads, [X,W,b]) )

    # Log periodically.
    if iter % 20 == 0:
        print(f"Training loss at iteration {iter}: {cost_value:0.1f}")

<a name="6"></a>
## 6 - คำแนะนำ (Recommendations)
ด้านล่าง เราคำนวณเรตติ้งสำหรับภาพยนตร์และผู้ใช้ทั้งหมด และแสดงภาพยนตร์ที่แนะนำ เรตติ้งเหล่านี้ยึดตามภาพยนตร์และเรตติ้งที่ป้อนเป็น `my_ratings[]` ข้างต้น เพื่อทำนายเรตติ้งของภาพยนตร์ $i$ สำหรับผู้ใช้  $j$, คุณคำนวณ $\mathbf{w}^{(j)} \cdot \mathbf{x}^{(i)} + b^{(j)}$. ซึ่งสามารถคำนวณได้สำหรับทุกเรตติ้งโดยใช้การคูณเมทริกซ์

In [ ]:
# Make a prediction using trained weights and biases
p = np.matmul(X.numpy(), np.transpose(W.numpy())) + b.numpy()

#restore the mean
pm = p + Ymean

my_predictions = pm[:,0]

# sort predictions
ix = tf.argsort(my_predictions, direction='DESCENDING')

for i in range(17):
    j = ix[i]
    if j not in my_rated:
        print(f'Predicting rating {my_predictions[j]:0.2f} for movie {movieList[j]}')

print('\n\nOriginal vs Predicted ratings:\n')
for i in range(len(my_ratings)):
    if my_ratings[i] > 0:
        print(f'Original {my_ratings[i]}, Predicted {my_predictions[i]:0.2f} for {movieList[i]}')

ในทางปฏิบัติ ข้อมูลเพิ่มเติมสามารถนำมาใช้เพื่อปรับปรุงการคาดการณ์ของเราได้ จากตัวอย่างข้างต้น คะแนนที่คาดการณ์สำหรับภาพยนตร์ไม่กี่ร้อยเรื่องแรกอยู่ในช่วงที่แคบ เราสามารถปรับปรุงข้อมูลข้างต้นได้โดยการเลือกจากภาพยนตร์ยอดนิยมเหล่านั้น ซึ่งเป็นภาพยนตร์ที่มีคะแนนเฉลี่ยสูงและมีผู้ให้คะแนนมากกว่า 20 คน ส่วนนี้ใช้ดาต้าเฟรมของ[Pandas](https://pandas.pydata.org/) ซึ่งมีฟีเจอร์การจัดเรียงข้อมูลที่สะดวกมากมาย

In [ ]:
filter=(movieList_df["number of ratings"] > 20)
movieList_df["pred"] = my_predictions
movieList_df = movieList_df.reindex(columns=["pred", "mean rating", "number of ratings", "title"])
movieList_df.loc[ix[:300]].loc[filter].sort_values("mean rating", ascending=False)

<a name="7"></a>
## 7 - ยินดีด้วย! <img align="left" src="./images/film_award.png"     style=" width:40px;  " >
คุณได้สร้างระบบแนะนำสินค้าที่มีประโยชน์แล้ว!